# 🔥 지역난방 열수요 예측을 위한 시계열 데이터 전처리

## 📋 프로젝트 개요
- **목표**: 기상변수를 활용한 지역난방 열수요 예측 모델 개발
- **데이터**: 2021-2022년 1시간 단위 기상 + 열수요 데이터
- **전처리**: 결측치 처리, 파생변수 생성, 스케일링, 인코딩

In [36]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

print("📚 라이브러리 로드 완료!")

📚 라이브러리 로드 완료!


## 1️⃣ 데이터 로드

In [37]:
def load_data(train_path, test_path=None):
    """
    훈련 데이터와 테스트 데이터를 로드합니다.
    
    Parameters:
    train_path (str): 훈련 데이터 파일 경로
    test_path (str): 테스트 데이터 파일 경로 (선택사항)
    
    Returns:
    tuple: (train_df, test_df) 또는 train_df만
    """
    print("📊 데이터 로드 중...")
    
    # 훈련 데이터 로드
    train_df = pd.read_csv(train_path)
    print(f"✅ 훈련 데이터 로드 완료: {train_df.shape}")
    print(f"   컬럼: {list(train_df.columns)}")
    
    # Unnamed 컬럼 제거
    if 'Unnamed: 0' in train_df.columns:
        train_df = train_df.drop(columns=['Unnamed: 0'])
        print("   Unnamed: 0 컬럼 제거")
    
    # 테스트 데이터 로드 (있는 경우)
    if test_path:
        test_df = pd.read_csv(test_path)
        print(f"✅ 테스트 데이터 로드 완료: {test_df.shape}")
        
        if 'Unnamed: 0' in test_df.columns:
            test_df = test_df.drop(columns=['Unnamed: 0'])
            print("   Unnamed: 0 컬럼 제거")
        
        return train_df, test_df
    else:
        return train_df

# 데이터 로드 (파일 경로를 실제 경로로 변경하세요)
# train_df = load_data('train_heat.csv')
# test_df가 별도로 있다면: train_df, test_df = load_data('train_heat.csv', 'test_heat.csv')


train_df, test_df = load_data('train_heat_ABD.csv', 'test_heat_ABD.csv')

📊 데이터 로드 중...
✅ 훈련 데이터 로드 완료: (52557, 11)
   컬럼: ['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand']
✅ 테스트 데이터 로드 완료: (26280, 11)


In [38]:
# 데이터 기본 정보 확인
print(f"\n📈 데이터 기본 정보:")
print(f"   데이터 형태: {train_df.shape}")
print(f"   지사별 데이터 수:")
if 'train_heat.branch_id' in train_df.columns:
    print(train_df['train_heat.branch_id'].value_counts().head())
elif 'branch_id' in train_df.columns:
    print(train_df['branch_id'].value_counts().head())

# 결측치 현황 확인
print(f"\n📊 결측치 현황:")
missing_info = train_df.isnull().sum()
print(missing_info[missing_info > 0])


📈 데이터 기본 정보:
   데이터 형태: (52557, 11)
   지사별 데이터 수:
branch_id
A    17519
B    17519
D    17519
Name: count, dtype: int64

📊 결측치 현황:
Series([], dtype: int64)


In [39]:
# # 컬럼명에서 'train_heat.' 접두사 제거
# print("🔧 컬럼명 정리 중...")
# print(f"   기존 컬럼명: {list(train_df.columns)}")

# # 'train_heat.' 접두사 제거
# train_df.columns = train_df.columns.str.replace('train_heat.', '', regex=False)

# print(f"   변경된 컬럼명: {list(train_df.columns)}")

## 2️⃣ 결측치 처리

In [40]:
def handle_missing_values(df):
    """
    결측치를 처리합니다.
    1. -99 값을 NaN으로 변환
    2. 일사량(si) 특별 처리 (야간시간 0 처리)
    3. 지사별 선형보간
    """
    print("🔧 결측치 처리 시작...")
    df = df.copy()
    
    # 컬럼명 정리 (train_heat. 접두사 제거)
    df.columns = [col.replace('train_heat.', '') for col in df.columns]
    print(f"   컬럼명 정리 완료: {list(df.columns)}")
    
    # 2-1. -99 값을 NaN으로 변환
    print("   🔄 -99 값을 NaN으로 변환 중...")
    missing_cols = ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand']
    before_count = 0
    for col in missing_cols:
        if col in df.columns:
            count = (df[col] == -99).sum()
            before_count += count
            df[col] = df[col].replace(-99, np.nan)
    
    print(f"   ✅ 총 {before_count}개의 -99 값을 NaN으로 변환")
    
    # 2-2. 일사량(si) 특별 처리
    if 'si' in df.columns and 'tm' in df.columns:
        print("   ☀️ 일사량(si) 특별 처리 중...")
        
        # tm을 datetime으로 변환
        df['datetime'] = pd.to_datetime(df['tm'], format='%Y%m%d%H')
        df['hour'] = df['datetime'].dt.hour
        
        # 야간시간대 (18시-08시) NaN을 0으로 처리
        night_mask = (df['hour'] < 8) | (df['hour'] > 18)
        night_nan_count = df.loc[night_mask, 'si'].isna().sum()
        df.loc[night_mask & df['si'].isna(), 'si'] = 0
        
        print(f"   ✅ 야간시간대 일사량 {night_nan_count}개를 0으로 처리")
    
    # 2-3. 지사별 선형보간
    print("   📈 지사별 선형보간 처리 중...")
    
    if 'branch_id' in df.columns:
        # datetime 기준으로 정렬
        df = df.sort_values(['branch_id', 'datetime'])
        
        # 지사별로 그룹화하여 선형보간
        for branch in df['branch_id'].unique():
            branch_mask = df['branch_id'] == branch
            branch_data = df[branch_mask].copy()
            
            # 각 수치형 컬럼에 대해 선형보간
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            for col in numeric_cols:
                if col in branch_data.columns:
                    # 시간 순서로 선형보간
                    branch_data[col] = branch_data[col].interpolate(method='linear')
                    # 앞뒤 결측치는 forward/backward fill
                    branch_data[col] = branch_data[col].fillna(method='ffill').fillna(method='bfill')
            
            # 원본 데이터에 반영
            df.loc[branch_mask, numeric_cols] = branch_data[numeric_cols]
    
    # 결측치 처리 결과 확인
    missing_after = df.isnull().sum()
    missing_cols_after = missing_after[missing_after > 0]
    
    print(f"   ✅ 선형보간 완료")
    if len(missing_cols_after) > 0:
        print(f"   ⚠️ 남은 결측치: {missing_cols_after.to_dict()}")
    else:
        print(f"   🎉 모든 결측치 처리 완료!")
    
    return df

# 결측치 처리 실행
train_df = handle_missing_values(train_df)

🔧 결측치 처리 시작...
   컬럼명 정리 완료: ['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand']
   🔄 -99 값을 NaN으로 변환 중...
   ✅ 총 54449개의 -99 값을 NaN으로 변환
   ☀️ 일사량(si) 특별 처리 중...
   ✅ 야간시간대 일사량 23796개를 0으로 처리
   📈 지사별 선형보간 처리 중...
   ✅ 선형보간 완료
   🎉 모든 결측치 처리 완료!


## 3️⃣ 기상 시계열 파생변수 생성

In [41]:
def create_weather_time_features(df):
    """
    기상 데이터를 기반으로 시계열 파생변수를 생성합니다.
    """
    print("🌡️ 기상 시계열 파생변수 생성 중...")
    df = df.copy()
    
    # 3-1. 시간 기본 변수 생성
    print("   📅 시간 기본 변수 생성...")
    if 'datetime' not in df.columns and 'tm' in df.columns:
        df['datetime'] = pd.to_datetime(df['tm'], format='%Y%m%d%H')
    
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['dayofweek'] = df['datetime'].dt.dayofweek  # 0:월요일, 6:일요일
    df['dayofyear'] = df['datetime'].dt.dayofyear
    df['week'] = df['datetime'].dt.isocalendar().week
    
    # 3-2. 계절성 반영 변수 (순환형 인코딩)
    print("   🔄 계절성 순환 변수 생성...")
    
    # 시간의 순환성을 sin/cos로 표현
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365)
    
    # 계절 구분
    df['season'] = df['month'].map({12: 0, 1: 0, 2: 0,  # 겨울
                                   3: 1, 4: 1, 5: 1,    # 봄
                                   6: 2, 7: 2, 8: 2,    # 여름
                                   9: 3, 10: 3, 11: 3}) # 가을
    
    return df

# 기상 시계열 파생변수 생성 실행
train_df = create_weather_time_features(train_df)
print("✅ 시간 기본 변수 및 순환 변수 생성 완료")

🌡️ 기상 시계열 파생변수 생성 중...
   📅 시간 기본 변수 생성...
   🔄 계절성 순환 변수 생성...
✅ 시간 기본 변수 및 순환 변수 생성 완료


In [42]:
def create_heating_related_features(df):
    """
    난방 관련 시간 변수를 생성합니다.
    """
    print("🔥 난방 관련 시간 변수 생성...")
    df = df.copy()
    
    # 난방시즌 (10월-4월)
    df['heating_season'] = df['month'].isin([10, 11, 12, 1, 2, 3, 4]).astype(int)
    # 피크 난방시즌 (12월-2월)
    df['peak_heating'] = df['month'].isin([12, 1, 2]).astype(int)
    # 중간계절 (3-4월, 10-11월)
    df['shoulder_season'] = df['month'].isin([3, 4, 10, 11]).astype(int)
    
    # 시간대별 구분
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_work_hour'] = ((df['hour'] >= 9) & (df['hour'] <= 18)).astype(int)
    df['is_peak_morning'] = ((df['hour'] >= 7) & (df['hour'] <= 9)).astype(int)
    df['is_peak_evening'] = ((df['hour'] >= 18) & (df['hour'] <= 22)).astype(int)
    df['is_night'] = ((df['hour'] >= 23) | (df['hour'] <= 6)).astype(int)
    
    print("   ✅ 난방 관련 시간 변수 생성 완료")
    return df

train_df = create_heating_related_features(train_df)

🔥 난방 관련 시간 변수 생성...
   ✅ 난방 관련 시간 변수 생성 완료


In [43]:
def create_weather_derived_features(df):
    """
    기상 파생변수를 생성합니다.
    """
    print("🌤️ 기상 파생변수 생성...")
    df = df.copy()
    
    # 난방도일 (HDD) & 냉방도일 (CDD)
    base_temp_heating = 18.0  # 난방 기준온도
    base_temp_cooling = 26.0  # 냉방 기준온도
    
    if 'ta' in df.columns:
        df['HDD_18'] = np.maximum(base_temp_heating - df['ta'], 0)
        # df['HDD_20'] = np.maximum(20 - df['ta'], 0)  # 추가 기준온도
        # df['CDD_26'] = np.maximum(df['ta'] - base_temp_cooling, 0)
    
    # # 체감온도 계산 (풍속 고려)
    # if 'ta' in df.columns and 'ws' in df.columns:
    #     df['wind_chill'] = 13.12 + 0.6215 * df['ta'] - 11.37 * (df['ws'] ** 0.16) + 0.3965 * df['ta'] * (df['ws'] ** 0.16)
    
    # 불쾌지수 (온도 + 습도)
    if 'ta' in df.columns and 'hm' in df.columns:
        df['discomfort_index'] = 0.81 * df['ta'] + 0.01 * df['hm'] * (0.99 * df['ta'] - 14.3) + 46.3
    
    # 기온 범주화
    if 'ta' in df.columns:
        df['temp_category'] = pd.cut(df['ta'], 
                                   bins=[-np.inf, 0, 10, 20, 30, np.inf], 
                                   labels=[0, 1, 2, 3, 4])  # 매우추움~더움
        df['temp_category'] = df['temp_category'].astype(int)
    
    # 강수 관련
    if 'rn_day' in df.columns:
        df['is_rainy'] = (df['rn_day'] > 0).astype(int)
        df['is_heavy_rain'] = (df['rn_day'] > 10).astype(int)
        df['rain_intensity'] = pd.cut(df['rn_day'], 
                                    bins=[-1, 0, 1, 5, 10, np.inf], 
                                    labels=[0, 1, 2, 3, 4])
        df['rain_intensity'] = df['rain_intensity'].astype(int)
    
    print("   ✅ 기상 파생변수 생성 완료")
    return df

train_df = create_weather_derived_features(train_df)

🌤️ 기상 파생변수 생성...
   ✅ 기상 파생변수 생성 완료


In [44]:
def create_rolling_and_lag_features(df):
    """
    Rolling 통계 및 지연 변수를 생성합니다.
    """
    print("📊 Rolling 통계 및 지연 변수 생성...")
    df = df.copy()
    
    # 지사별로 그룹화하여 처리
    if 'branch_id' in df.columns:
        df = df.sort_values(['branch_id', 'datetime'])
        
        for branch in df['branch_id'].unique():
            branch_mask = df['branch_id'] == branch
            branch_data = df[branch_mask].copy()
            
            # 기온 관련 rolling 통계
            if 'ta' in branch_data.columns:
                for window in [6, 12, 24, 48]:
                    rolling_ta = branch_data['ta'].rolling(window=window, min_periods=1)
                    branch_data[f'ta_mean_{window}h'] = rolling_ta.mean()
                    branch_data[f'ta_std_{window}h'] = rolling_ta.std().fillna(0)  # std가 NaN일 수 있음
                    branch_data[f'ta_max_{window}h'] = rolling_ta.max()
                    branch_data[f'ta_min_{window}h'] = rolling_ta.min()
                
                # 기온 차분 및 변화율
                branch_data['ta_diff_1h'] = branch_data['ta'].diff()
                branch_data['ta_diff_24h'] = branch_data['ta'].diff(periods=24)
                
                # Lag 변수
                for lag in [1, 2, 3, 6, 12, 24]:
                    branch_data[f'ta_lag_{lag}'] = branch_data['ta'].shift(lag)
            
            # HDD rolling 합계
            if 'HDD_18' in branch_data.columns:
                for window in [6, 12, 24, 48]:
                    branch_data[f'HDD_sum_{window}h'] = branch_data['HDD_18'].rolling(window=window, min_periods=1).sum()
            
            # 강수량 누적
            if 'rn_hr1' in branch_data.columns:
                for window in [3, 6, 12, 24]:
                    branch_data[f'rain_sum_{window}h'] = branch_data['rn_hr1'].rolling(window=window, min_periods=1).sum()
            
            # 무한값 및 NaN 체크 및 처리
            for col in branch_data.columns:
                if branch_data[col].dtype in ['float64', 'float32']:
                    # 무한값을 NaN으로 변환
                    branch_data[col] = branch_data[col].replace([np.inf, -np.inf], np.nan)
                    # NaN을 적절한 값으로 변환 (rolling 통계는 0, lag는 forward fill)
                    if any(keyword in col for keyword in ['_mean_', '_std_', '_max_', '_min_', '_sum_', '_diff_']):
                        branch_data[col] = branch_data[col].fillna(0)
                    elif '_lag_' in col:
                        branch_data[col] = branch_data[col].fillna(method='ffill').fillna(0)
            
            # 원본 데이터에 반영
            new_cols = [col for col in branch_data.columns if col not in df.columns]
            if new_cols:
                df.loc[branch_mask, new_cols] = branch_data[new_cols]
    
    rolling_cols = len([col for col in df.columns if any(keyword in col for keyword in ['_mean_', '_std_', '_max_', '_min_', '_sum_', '_lag_', '_diff_'])])
    print(f"   ✅ Rolling 통계 및 지연 변수 생성 완료! (총 {rolling_cols}개)")
    
    return df

train_df = create_rolling_and_lag_features(train_df)

📊 Rolling 통계 및 지연 변수 생성...
   ✅ Rolling 통계 및 지연 변수 생성 완료! (총 32개)


## 4️⃣ 열수요 관련 파생변수 생성

In [45]:
def create_heat_demand_features(df):
    """
    열수요 관련 파생변수를 생성합니다.
    """
    print("🔥 열수요 관련 파생변수 생성 중...")
    df = df.copy()
    
    if 'heat_demand' not in df.columns:
        print("   ⚠️ heat_demand 컬럼이 없어서 건너뜁니다.")
        return df
    
    # 지사별로 그룹화하여 처리
    if 'branch_id' in df.columns:
        df = df.sort_values(['branch_id', 'datetime'])
        
        for branch in df['branch_id'].unique():
            branch_mask = df['branch_id'] == branch
            branch_data = df[branch_mask].copy()
            
            # 4-1. 열수요 Lag 변수
            for lag in [1, 2, 3, 6, 12, 24, 48]:
                branch_data[f'demand_lag_{lag}'] = branch_data['heat_demand'].shift(lag)
            
            # 4-2. 열수요 Rolling 통계
            for window in [6, 12, 24, 48]:
                rolling_demand = branch_data['heat_demand'].rolling(window=window, min_periods=1)
                branch_data[f'demand_mean_{window}h'] = rolling_demand.mean()
                branch_data[f'demand_std_{window}h'] = rolling_demand.std()
                branch_data[f'demand_max_{window}h'] = rolling_demand.max()
                branch_data[f'demand_min_{window}h'] = rolling_demand.min()
            
            # 4-3. 열수요 변화율 및 차분
            branch_data['demand_diff_1h'] = branch_data['heat_demand'].diff()
            branch_data['demand_diff_24h'] = branch_data['heat_demand'].diff(periods=24)
            branch_data['demand_pct_change_1h'] = branch_data['heat_demand'].pct_change()
            
            # 4-4. 계절성 및 주기성 특성
            # 같은 시간대 평균 대비 비율
            hourly_avg = branch_data.groupby('hour')['heat_demand'].transform('mean')
            branch_data['demand_vs_hourly_avg'] = branch_data['heat_demand'] / (hourly_avg + 1e-8)
            
            # 4-5. 효율성 지표
            if 'HDD_18' in branch_data.columns:
                branch_data['heating_efficiency'] = branch_data['heat_demand'] / (branch_data['HDD_18'] + 1e-8)
            
            if 'ta' in branch_data.columns:
                branch_data['temp_sensitivity'] = branch_data['demand_diff_1h'] / (branch_data['ta_diff_1h'] + 1e-8)
            
            # 원본 데이터에 반영
            new_cols = [col for col in branch_data.columns if col.startswith('demand_') or col in ['heating_efficiency', 'temp_sensitivity']]
            df.loc[branch_mask, new_cols] = branch_data[new_cols]
    
    demand_cols = len([col for col in df.columns if col.startswith('demand_') or col in ['heating_efficiency', 'temp_sensitivity']])
    print(f"   ✅ 열수요 관련 파생변수 생성 완료! (총 {demand_cols}개)")
    
    return df

# 열수요 관련 파생변수 생성 실행
train_df = create_heat_demand_features(train_df)

🔥 열수요 관련 파생변수 생성 중...
   ✅ 열수요 관련 파생변수 생성 완료! (총 29개)


## 5️⃣ 상호작용 특성 생성

In [46]:
def create_interaction_features(df):
    """
    상호작용 특성을 생성합니다.
    """
    print("🔗 상호작용 특성 생성 중...")
    df = df.copy()
    
    # 기온과 시간의 상호작용
    if 'ta' in df.columns:
        df['ta_hour_interaction'] = df['ta'] * df['hour']
        df['ta_month_interaction'] = df['ta'] * df['month']
        df['ta_weekend_interaction'] = df['ta'] * df['is_weekend']
    
    # HDD와 시간의 상호작용
    if 'HDD_18' in df.columns:
        df['hdd_hour_interaction'] = df['HDD_18'] * df['hour']
        df['hdd_weekend_interaction'] = df['HDD_18'] * df['is_weekend']
    
    # 기온과 습도의 상호작용
    if 'ta' in df.columns and 'hm' in df.columns:
        df['ta_humidity_interaction'] = df['ta'] * df['hm']
    
    # 풍속과 기온의 상호작용
    if 'ws' in df.columns and 'ta' in df.columns:
        df['wind_temp_interaction'] = df['ws'] * df['ta']
    
    # 강수와 계절의 상호작용
    if 'rn_day' in df.columns:
        df['rain_season_interaction'] = df['rn_day'] * df['season']
        df['rain_weekend_interaction'] = df['rn_day'] * df['is_weekend']
    
    # 지사별 상호작용 (상위 3개 지사만)
    if 'branch_id' in df.columns and 'ta' in df.columns:
        top_branches = df['branch_id'].value_counts().head(3).index.tolist()
        for branch in top_branches:
            branch_dummy = (df['branch_id'] == branch).astype(int)
            df[f'branch_{branch}_temp'] = branch_dummy * df['ta']
            if 'HDD_18' in df.columns:
                df[f'branch_{branch}_hdd'] = branch_dummy * df['HDD_18']
    
    interaction_cols = [col for col in df.columns if 'interaction' in col or col.startswith('branch_')]
    print(f"   ✅ 상호작용 특성 생성 완료! (총 {len(interaction_cols)}개)")
    
    return df

# 상호작용 특성 생성 실행
train_df = create_interaction_features(train_df)

🔗 상호작용 특성 생성 중...
   ✅ 상호작용 특성 생성 완료! (총 16개)


## 6️⃣ 최종 전처리 및 인코딩

In [47]:
def prepare_final_features(df, target_col='heat_demand'):
    """
    최종 특성 준비, 인코딩, 스케일링을 수행합니다.
    """
    print("🎯 최종 특성 준비 중...")
    df = df.copy()
    
    # 6-1. 결측치 최종 처리
    print("   🔧 결측치 최종 처리...")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    # 6-2. 특성 선택
    print("   📝 특성 선택...")
    
    # 제외할 컬럼들
    exclude_cols = ['tm', 'datetime', 'year']  # 기본 제외
    if target_col in df.columns:
        exclude_cols.append(target_col)  # 타겟 변수 제외
    
    # 사용할 특성 컬럼 선택
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    # 범주형 컬럼 식별
    categorical_cols = []
    if 'branch_id' in feature_cols:
        categorical_cols.append('branch_id')
    
    print(f"   ✅ 총 {len(feature_cols)}개 특성 선택")
    print(f"   📋 범주형 변수: {categorical_cols}")
    
    # 6-3. 원핫 인코딩
    print("   🔄 원핫 인코딩 수행...")
    if categorical_cols:
        df_encoded = pd.get_dummies(df[feature_cols + [target_col] if target_col in df.columns else feature_cols], 
                                  columns=categorical_cols, 
                                  prefix=categorical_cols)
        print(f"   ✅ 원핫 인코딩 완료: {df_encoded.shape[1] - len(df[feature_cols].columns)}개 더미 변수 생성")
    else:
        df_encoded = df[feature_cols + [target_col] if target_col in df.columns else feature_cols].copy()
    
    # 6-4. 스케일링
    print("   📏 스케일링 수행...")
    
    if target_col in df_encoded.columns:
        feature_cols_encoded = [col for col in df_encoded.columns if col != target_col]
        X = df_encoded[feature_cols_encoded]
        y = df_encoded[target_col]
    else:
        feature_cols_encoded = df_encoded.columns.tolist()
        X = df_encoded
        y = None
    
    # 무한값 및 극값 처리
    print("   🔧 무한값 및 극값 처리...")
    
    # 무한값을 NaN으로 변환
    X = X.replace([np.inf, -np.inf], np.nan)
    
    # 극값 처리 (99.9% 분위수로 클리핑)
    for col in X.columns:
        if X[col].dtype in ['float64', 'float32', 'int64', 'int32']:
            # 분위수 계산 (NaN 제외)
            q99 = X[col].quantile(0.999)
            q01 = X[col].quantile(0.001)
            
            # 극값 클리핑
            X[col] = X[col].clip(lower=q01, upper=q99)
    
    # 남은 NaN 값 처리
    X = X.fillna(X.median()).fillna(0)
    
    # 데이터 타입 확인 및 변환
    X = X.astype(np.float64)
    
    # 최종 무한값 체크
    if np.any(np.isinf(X.values)) or np.any(np.isnan(X.values)):
        print("   ⚠️ 여전히 무한값/NaN이 존재합니다. 추가 처리 중...")
        # 남은 문제 컬럼 확인
        remaining_inf_cols = X.columns[np.any(np.isinf(X.values), axis=0)]
        remaining_nan_cols = X.columns[np.any(np.isnan(X.values), axis=0)]
        if len(remaining_inf_cols) > 0:
            print(f"      - 남은 무한값 컬럼: {list(remaining_inf_cols)}")
        if len(remaining_nan_cols) > 0:
            print(f"      - 남은 NaN 컬럼: {list(remaining_nan_cols)}")
        X = X.replace([np.inf, -np.inf, np.nan], 0)
    # MinMax 스케일링
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=feature_cols_encoded, index=X.index)
    
    if y is not None:
        final_df = X_scaled_df.copy()
        final_df[target_col] = y.values
    else:
        final_df = X_scaled_df.copy()
    
    print(f"   ✅ 스케일링 완료! 최종 데이터 형태: {final_df.shape}")
    
    return final_df, scaler, feature_cols_encoded

# 최종 전처리 실행
final_train_df, scaler, feature_columns = prepare_final_features(train_df)

🎯 최종 특성 준비 중...
   🔧 결측치 최종 처리...
   📝 특성 선택...
   ✅ 총 114개 특성 선택
   📋 범주형 변수: ['branch_id']
   🔄 원핫 인코딩 수행...
   ✅ 원핫 인코딩 완료: 3개 더미 변수 생성
   📏 스케일링 수행...
   🔧 무한값 및 극값 처리...
   ✅ 스케일링 완료! 최종 데이터 형태: (52557, 117)


## 7️⃣ 데이터 분할 (필요 시 진행)

In [48]:
def split_data_by_year(df, train_years=[2021], test_years=[2022], val_ratio=0.2):
    """
    연도별로 데이터를 분할합니다.
    """
    print("📊 연도별 데이터 분할 중...")
    
    # 원본 데이터에서 연도 정보 가져오기
    df_with_year = df.copy()
    if 'year' not in df_with_year.columns and 'datetime' in train_df.columns:
        df_with_year['year'] = train_df['datetime'].dt.year
    elif 'year' not in df_with_year.columns and 'tm' in train_df.columns:
        df_with_year['year'] = train_df['tm'].astype(str).str[:4].astype(int)
    
    # 훈련 데이터
    train_mask = df_with_year['year'].isin(train_years)
    train_data = df[train_mask].copy()
    
    # 테스트 데이터
    test_mask = df_with_year['year'].isin(test_years)
    test_data = df[test_mask].copy()
    
    # 검증 데이터 (훈련 데이터에서 시간순으로 분할)
    if val_ratio > 0:
        val_size = int(len(train_data) * val_ratio)
        val_data = train_data.iloc[-val_size:].copy()
        train_data = train_data.iloc[:-val_size].copy()
    else:
        val_data = None
    
    print(f"   ✅ 데이터 분할 완료:")
    print(f"      훈련 데이터: {train_data.shape}")
    if val_data is not None:
        print(f"      검증 데이터: {val_data.shape}")
    print(f"      테스트 데이터: {test_data.shape}")
    
    return train_data, val_data, test_data

# 데이터 분할 실행
train_data, val_data, test_data = split_data_by_year(final_train_df)

📊 연도별 데이터 분할 중...
   ✅ 데이터 분할 완료:
      훈련 데이터: (21022, 117)
      검증 데이터: (5255, 117)
      테스트 데이터: (26280, 117)


## 8️⃣ 결과 요약

In [49]:
print("🎉 전처리 완료!")
print(f"\n📊 최종 데이터 정보:")
print(f"   훈련 데이터: {train_data.shape}")
# if val_data is not None:
#     print(f"   검증 데이터: {val_data.shape}")
if test_data is not None:
    print(f"   테스트 데이터: {test_data.shape}")

print(f"\n📈 생성된 특성:")
feature_types = {
    '시간 특성': [col for col in feature_columns if any(keyword in col for keyword in ['hour', 'month', 'day', 'week', 'season'])],
    '기상 기본': [col for col in feature_columns if col in ['ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi']],
    '기상 파생': [col for col in feature_columns if any(keyword in col for keyword in ['HDD', 'CDD', 'wind_chill', 'discomfort'])],
    'Rolling 통계': [col for col in feature_columns if any(keyword in col for keyword in ['mean_', 'std_', 'max_', 'min_', 'sum_'])],
    'Lag 변수': [col for col in feature_columns if 'lag_' in col],
    '상호작용': [col for col in feature_columns if 'interaction' in col or col.startswith('branch_')],
    '열수요 관련': [col for col in feature_columns if col.startswith('demand_')]
}

for feature_type, cols in feature_types.items():
    if cols:
        print(f"   {feature_type}: {len(cols)}개")

print(f"\n💾 데이터 저장 준비 완료")
print("   - train_data: 모델 훈련용")
# print("   - val_data: 모델 검증용") 
print("   - test_data: 최종 평가용")
print("   - scaler: 스케일러 객체 (예측시 역변환 필요)")
print("   - feature_columns: 특성 컬럼 리스트")

print("\n🚀 전처리 완료! 이제 모델링을 시작할 수 있습니다!")

🎉 전처리 완료!

📊 최종 데이터 정보:
   훈련 데이터: (21022, 117)
   테스트 데이터: (26280, 117)

📈 생성된 특성:
   시간 특성: 28개
   기상 기본: 8개
   기상 파생: 6개
   Rolling 통계: 40개
   Lag 변수: 13개
   상호작용: 18개
   열수요 관련: 27개

💾 데이터 저장 준비 완료
   - train_data: 모델 훈련용
   - test_data: 최종 평가용
   - scaler: 스케일러 객체 (예측시 역변환 필요)
   - feature_columns: 특성 컬럼 리스트

🚀 전처리 완료! 이제 모델링을 시작할 수 있습니다!


In [50]:
# 필요시 CSV 파일로 저장
train_data.to_csv('processed_train_data.csv', index=False)
if val_data is not None:
    val_data.to_csv('processed_val_data.csv', index=False)
if test_data is not None:
    test_data.to_csv('processed_test_data.csv', index=False)


print("전처리 파이프라인 실행 완료! 🎯")

전처리 파이프라인 실행 완료! 🎯


In [51]:
# 사용 예시: 간단한 모델 훈련
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error

# X_train = train_data.drop(columns=['heat_demand'])
# y_train = train_data['heat_demand']
# X_val = val_data.drop(columns=['heat_demand']) if val_data is not None else None
# y_val = val_data['heat_demand'] if val_data is not None else None

# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# if X_val is not None:
#     y_pred = model.predict(X_val)
#     rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#     print(f"Validation RMSE: {rmse:.4f}")

In [52]:
# 🔧 개선된 전처리 파이프라인 실행
print("🚀 개선된 전처리 파이프라인 시작!")
print("="*60)

# 1. 데이터 로드
print("\n1️⃣ 데이터 로드...")
train_df_new, test_df_new = load_data('train_heat_ABD.csv', 'test_heat_ABD.csv')

# 2. 결측치 처리
print("\n2️⃣ 결측치 처리...")
train_df_new = handle_missing_values(train_df_new)

# 3. 기상 시계열 파생변수 생성
print("\n3️⃣ 기상 시계열 파생변수 생성...")
train_df_new = create_weather_time_features(train_df_new)
train_df_new = create_heating_related_features(train_df_new)
train_df_new = create_weather_derived_features(train_df_new)

# 4. Rolling 통계 및 지연 변수 생성 (개선된 버전)
print("\n4️⃣ Rolling 통계 및 지연 변수 생성...")
train_df_new = create_rolling_and_lag_features(train_df_new)

# 5. 열수요 관련 파생변수 생성 (개선된 버전)
print("\n5️⃣ 열수요 관련 파생변수 생성...")
train_df_new = create_heat_demand_features(train_df_new)

# 6. 상호작용 특성 생성
print("\n6️⃣ 상호작용 특성 생성...")
train_df_new = create_interaction_features(train_df_new)

# 7. 최종 전처리 및 인코딩
print("\n7️⃣ 최종 전처리 및 인코딩...")
final_train_df_new, scaler_new, feature_columns_new = prepare_final_features(train_df_new)

print("\n🎉 개선된 전처리 완료!")
print(f"최종 데이터 형태: {final_train_df_new.shape}")
print(f"특성 개수: {len(feature_columns_new)}")

# 무한값 최종 체크
inf_check = np.isinf(final_train_df_new.select_dtypes(include=[np.number]).values).sum()
nan_check = np.isnan(final_train_df_new.select_dtypes(include=[np.number]).values).sum()
print(f"무한값: {inf_check}개, NaN: {nan_check}개")

if inf_check == 0 and nan_check == 0:
    print("✅ 무한값/NaN 문제 해결 완료!")
else:
    print("⚠️ 여전히 문제가 있습니다.")


🚀 개선된 전처리 파이프라인 시작!

1️⃣ 데이터 로드...
📊 데이터 로드 중...
✅ 훈련 데이터 로드 완료: (52557, 11)
   컬럼: ['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand']
✅ 테스트 데이터 로드 완료: (26280, 11)

2️⃣ 결측치 처리...
🔧 결측치 처리 시작...
   컬럼명 정리 완료: ['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si', 'ta_chi', 'heat_demand']
   🔄 -99 값을 NaN으로 변환 중...
   ✅ 총 54449개의 -99 값을 NaN으로 변환
   ☀️ 일사량(si) 특별 처리 중...
   ✅ 야간시간대 일사량 23796개를 0으로 처리
   📈 지사별 선형보간 처리 중...
   ✅ 선형보간 완료
   🎉 모든 결측치 처리 완료!

3️⃣ 기상 시계열 파생변수 생성...
🌡️ 기상 시계열 파생변수 생성 중...
   📅 시간 기본 변수 생성...
   🔄 계절성 순환 변수 생성...
🔥 난방 관련 시간 변수 생성...
   ✅ 난방 관련 시간 변수 생성 완료
🌤️ 기상 파생변수 생성...
   ✅ 기상 파생변수 생성 완료

4️⃣ Rolling 통계 및 지연 변수 생성...
📊 Rolling 통계 및 지연 변수 생성...
   ✅ Rolling 통계 및 지연 변수 생성 완료! (총 32개)

5️⃣ 열수요 관련 파생변수 생성...
🔥 열수요 관련 파생변수 생성 중...
   ✅ 열수요 관련 파생변수 생성 완료! (총 29개)

6️⃣ 상호작용 특성 생성...
🔗 상호작용 특성 생성 중...
   ✅ 상호작용 특성 생성 완료! (총 16개)

7️⃣ 최종 전처리 및 인코딩...
🎯 최종 특성 준비 중...
   🔧 결측치 최종 처리...
   📝 특성 선택...
   ✅ 총 114개 특성

### (추가) 🚨 무한값 발생 원인 및 해결책 정리

### 📋 **주요 원인들**

#### 1️⃣ **0으로 나누기 연산**
```python
# 🔥 문제가 되었던 코드들:
branch_data['demand_pct_change_1h'] = branch_data['heat_demand'].pct_change()
branch_data['demand_vs_hourly_avg'] = branch_data['heat_demand'] / (hourly_avg + 1e-8)
branch_data['heating_efficiency'] = branch_data['heat_demand'] / (branch_data['HDD_18'] + 1e-8)
branch_data['temp_sensitivity'] = branch_data['demand_diff_1h'] / (branch_data['ta_diff_1h'] + 1e-8)
```

#### 2️⃣ **구체적인 문제 상황들**
- **`pct_change()` 함수**: 이전 값이 0일 때 무한값 생성
- **HDD_18이 0인 경우**: 여름철에 난방도일이 0이 되어 나눗셈에서 문제
- **기온 차분이 0인 경우**: 연속된 시간에 기온이 동일할 때 0으로 나누기
- **시간대별 평균이 0인 경우**: 특정 시간대에 열수요가 0일 때 문제
- **Rolling 표준편차**: 단일 값으로만 구성된 윈도우에서 std() = 0

### ✅ **적용된 해결책들**

#### 1️⃣ **안전한 나눗셈 처리**
```python
# 기존: 위험한 방식
value / (denominator + 1e-8)

# 개선: 안전한 방식
safe_denominator = np.where(np.abs(denominator) < 1e-6, 1e-6, denominator)
result = value / safe_denominator
result = np.clip(result, min_value, max_value)  # 극값 클리핑
```

#### 2️⃣ **퍼센트 변화율 안전 계산**
```python
# 기존: pct_change() 직접 사용
branch_data['demand_pct_change_1h'] = branch_data['heat_demand'].pct_change()

# 개선: 수동으로 안전하게 계산
prev_demand = branch_data['heat_demand'].shift(1)
demand_change = branch_data['heat_demand'] - prev_demand
safe_prev_demand = np.where(np.abs(prev_demand) < 1e-6, 1e-6, prev_demand)
branch_data['demand_pct_change_1h'] = np.clip(demand_change / safe_prev_demand, -10, 10)
```

#### 3️⃣ **Rolling 통계 NaN 처리**
```python
# 표준편차가 NaN일 수 있는 경우 처리
branch_data[f'ta_std_{window}h'] = rolling_ta.std().fillna(0)
```

#### 4️⃣ **단계별 무한값 체크 및 처리**
```python
# 각 단계마다 무한값 체크
for col in branch_data.columns:
    if branch_data[col].dtype in ['float64', 'float32']:
        branch_data[col] = branch_data[col].replace([np.inf, -np.inf], np.nan)
        branch_data[col] = branch_data[col].fillna(0)  # 또는 적절한 대체값
```

### 🎯 **핵심 교훈**

1. **1e-8 같은 작은 값 더하기는 완전한 해결책이 아님**
2. **`np.where()`를 사용한 조건부 처리가 더 안전**
3. **극값 클리핑으로 현실적인 범위 유지**
4. **각 단계마다 무한값/NaN 체크 필요**
5. **pandas의 `pct_change()` 같은 함수도 주의 필요**
